# Advicent Coding Challenge
## Savannah Evans
## 10/19/2019

The purpose of this notebook is to explore and clean the dataset.

In [1]:
import sys

# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline 
#this line above prepares the jupyter notebook for working with matplotlib
import numpy as np # imports a fast numerical programming library
import scipy as sp #imports stats functions, amongst other things
import matplotlib as mpl # this actually imports matplotlib
import matplotlib.cm as cm # allows us easy access to colormaps
import matplotlib.pyplot as plt # sets up plotting under plt
import pandas as pd #lets us handle data as dataframes
# sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns #sets up styles and gives us more plotting options

In [39]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.widgets import RangeSlider, Select, MultiSelect, DataTable, TableColumn
from bokeh.palettes import Colorblind
from bokeh.layouts import layout, column

In [91]:
all_data = pd.read_csv("college_costs.csv", encoding='mbcs')
print(all_data.head())
print(all_data.dtypes)

                      College  Tuition (in-state)  Tuition (out-of-state)  Room & Board
0          Adelphi University             38657.0                     NaN       15527.0
1         Agnes Scott College             54007.0                     NaN       12449.0
2  Albany College of Pharmacy             34154.0                     NaN       11274.0
3     Albertus Magnus College             32386.0                     NaN       13520.0
4           Alfred University             32525.0                     NaN       12763.0
College                    object
Tuition (in-state)        float64
Tuition (out-of-state)    float64
Room & Board              float64
dtype: object


## Clean the dataset

* Rename columns to a code friendly name.
* Ensure clean datset.
* Create new columns for application usage.

In [92]:
df = all_data.rename(columns={"College": "college", "Tuition (in-state)": "tuitionIn", "Tuition (out-of-state)": "tuitionOut", "Room & Board": "room"})
print(df.head())

                      college  tuitionIn  tuitionOut     room
0          Adelphi University    38657.0         NaN  15527.0
1         Agnes Scott College    54007.0         NaN  12449.0
2  Albany College of Pharmacy    34154.0         NaN  11274.0
3     Albertus Magnus College    32386.0         NaN  13520.0
4           Alfred University    32525.0         NaN  12763.0


In [93]:
# Ensure there are no college duplicates
print("College duplicates:", len(df.college) != len(df.college.unique()))
print("College field missing: ", len(df.college) != len(df.college.dropna()))

# Observe price ranges
min = df.tuitionIn.min()
max = df.tuitionIn.max()
print("Tuition in-state:", min, max, ", range:", max - min)

min = df.tuitionOut.min()
max = df.tuitionOut.max()
print("Tuition out-of-state:", min, max, ", range:", max - min)

# Observe missing values
arr = df["tuitionIn"]
print("Tuition in-state missing values:", len(arr) - len(arr.dropna()))

arr = df["tuitionOut"]
print("Tuition out-of-state missing values:", len(arr) - len(arr.dropna()))

arr = df["room"]
print("Room & board missing values:", len(arr) - len(arr.dropna()))

College duplicates: False
College field missing:  True
Tuition in-state: 5257.0 70260.0 , range: 65003.0
Tuition out-of-state: 11227.0 54967.0 , range: 43740.0
Tuition in-state missing values: 7
Tuition out-of-state missing values: 357
Room & board missing values: 1


In [95]:
# drop missing college names
df = df[df["college"].notnull()]

# Create new columns
df["totalIn"] = df["tuitionIn"] + df["room"]
df["totalOut"] = df["tuitionOut"] + df["room"]

## Application

### Requirements
The requirements for the application are as follows:
* Display Cost of the selected college.
* Option to specify tuition or tuition + room & Board.
* Option to select in-state or out-of-state.

Bonus
* Graph to compare colleges.
* Sliders to filter costs: tuition, room & board

In [101]:
def modify_doc(doc):
    
    # Define scatter plot and tools

    source = ColumnDataSource(data=dict(tuition=[], total=[], college=[], room=[]))

    TOOLTIPS=[
        ("College", "@college"),
        ("Tuition", "@tuition"),
        ("Room & Board", "@room"),
        ("Total", "@total")
    ]

    plot = figure(plot_width=300, plot_height=300, title="", toolbar_location=None, tooltips=TOOLTIPS, sizing_mode="scale_both")
    plot.circle(x="tuition", y="total", source=source, size=7, line_color=None)
    plot.xaxis.axis_label = "Tuition"
    plot.yaxis.axis_label = "Total"

    # Input Controls
    minT = df.tuitionOut.append(df.tuitionIn).min()
    maxT = df.tuitionOut.append(df.tuitionIn).max()
    minR = df["room"].min()
    maxR = df["room"].max()
    maxTot = maxT + maxR
    minTot = minT + minR
    
    total_slider = RangeSlider(title="Total", value=(minTot, maxTot), start=minTot, end=maxTot, step=500)
    room_slider = RangeSlider(title="Room & Board", value=(minR, maxR), start=minR, end=maxR, step=500)
    tuition_slider = RangeSlider(title="Tuition", value=(minT, maxT), start=minT, end=maxT, step=500)
    
    type_data= ["Tuition (in-state)", "Tuition (out-of-state)"]
    type_select = Select(title="Type", value="Tuition (in-state)", options=type_data)
    
    name_data = ["All"]+df["college"].tolist()
    college_select = MultiSelect(title="Colleges", options=name_data)

    def select_data(tuition, total):
        colleges = df["college"]
        if ("All" not in college_select.value):
            colleges = df["colleges"].isin(college_select.value)
        selected = df[
            (df["college"].isin(colleges)) &
            (df[tuition] >= tuition_slider.value[0]) &
            (df[tuition] <= tuition_slider.value[1]) &
            (df["room"] >= room_slider.value[0]) &
            (df["room"] <= room_slider.value[1]) &
            (df[total] >= total_slider.value[0]) &
            (df[total] <= total_slider.value[1])
        ]
        return selected

    def update(attr, old, new):
        type = type_select.value
        tuition_type = "tuitionIn"
        total = "totalIn"
        if (type == "Tuition (out-of-state)"):
            tuition_type = "tuitionOut"
            total = "totalOut"
        new_df = select_data(tuition_type, total)
        source.data = dict(
            tuition=new_df[tuition_type],
            total=new_df[total],
            college=new_df["college"],
            room=new_df["room"]
        )

    controls = [
        college_select,
        type_select,
        tuition_slider,
        room_slider,
        total_slider ]
    
    for control in controls:
        control.on_change('value', update)
    
    # Create table
    Columns = [TableColumn(field="college", title="College"),
              TableColumn(field="tuition", title="Tuition"),
              TableColumn(field="room", title="Room & Board"),
              TableColumn(field="total", title="Total")]
    
    table = DataTable(columns=Columns, source=source, sortable=True, height=500, width=500)
    
    inputs = column(*controls, width=200, height=400)
    inputs.sizing_mode = "fixed"

    college_select.value = ["All"] # Hack way to generate data
    
    l = layout([[inputs, plot], table], sizing_mode="scale_both")
    doc.add_root(l)

In [103]:
# Create views
output_notebook()
show(modify_doc)

Loading BokehJS ...

ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'MultiSelect', 'id': '5235'}, 'attr': 'value', 'new': ['Adelphi University']}], 'references': []}: KeyError('colleges')
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'MultiSelect', 'id': '5235'}, 'attr': 'value', 'new': ['Agnes Scott\xa0College']}], 'references': []}: KeyError('colleges')
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'MultiSelect', 'id': '5235'}, 'attr': 'value', 'new': ['Adelphi University']}], 'references': []}: KeyError('colleges')
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'MultiSelect